## LightGBM  (5)  Tuning
- player情報を付加
- 2017年の実績を付加
- 特徴量数: 135

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [9]:
ALL_MERGE = 'intermediate/all_merge_10.f'

In [10]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 150)

### 不要な列を削除

In [11]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [12]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [13]:
print(all_pitch.shape)
all_pitch.head()

(778767, 135)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,...,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,batter_cnt_bat,bat_game_cnt_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,firm,height,weight,draft_order,salary,foreigner,batter_cnt,bat_game_cnt,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,1,1,0,0,0.000000,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0,0,0,0,0.0,1.0,1,1,0,0,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0,0,0,0,0.0,1.0,1,1,0,0,0.383333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0,0,0,0,0.0,0.0,1,1,0,0,1.016667,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0,0,0,0,0.0,0.0,1,1,0,0,1.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [14]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 135)

### test
- 行数: 521650

In [15]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 135)

In [16]:
del all_pitch
gc.collect()

64

In [17]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [18]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [19]:
X_train, X_test, y_train, y_test = train_test_split(train_d, train['ball'])
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

### LGBM学習
- multiclass : クラス数=8

In [20]:
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
    }

### Tuningしながら学習

In [21]:
%%time
best_params, tuning_history = dict(), list()
lgb_model = lgb.train(lgb_param, lgb_train,
                      valid_sets=lgb_eval,
                      verbose_eval=0,
                      best_params=best_params,
                      tuning_history=tuning_history)
print('Best Params:', best_params)
#print('Tuning history:', tuning_history)

/home/yasunobu_umehara/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_num_leaves, val_score: 1.120758: 100%|##########| 20/20 [45:11<00:00, 135.59s/it]
tune_bagging_fraction_and_bagging_freq, val_score: 1.120758:  10%|#         | 1/10 [01:05<09:48, 65.38s/it][I 2020-05-06 03:43:49,511] Finished trial#0 with value: 1.1317480778335864 with parameters: {'bagging_fraction': 0.4283498874700205, 'bagging_freq': 5}. Best is trial#0 with value: 1.1317480778335864.
tune_bagging_fraction_and_bagging_freq, val_score: 1.120758:  20%|##        | 2/10 [02:28<09:25, 70.71s/it][I 2020-05-06 03:45:12,671] Finished trial#1 with value: 1.1222099805484382 with parameters: {'bagging_fraction': 0.7954305655366157, 'bagging_freq': 7}. Best is trial#1 with value: 1.1222099805484382.
tune_bagging_fraction_and_bagging_freq, val_score: 1.120758:  30%|###     

tune_bagging_fraction_and_bagging_freq, val_score: 1.120758:  40%|####      | 4/10 [05:27<08:03, 80.57s/it][I 2020-05-06 03:48:12,125] Finished trial#3 with value: 1.121730556575146 with parameters: {'bagging_fraction': 0.85485049099682, 'bagging_freq': 1}. Best is trial#2 with value: 1.1213881114040902.
tune_bagging_fraction_and_bagging_freq, val_score: 1.120758:  50%|#####     | 5/10 [06:42<06:33, 78.74s/it][I 2020-05-06 03:49:26,588] Finished trial#4 with value: 1.126630931921146 with parameters: {'bagging_fraction': 0.6074536136940337, 'bagging_freq': 7}. Best is trial#2 with value: 1.1213881114040902.
tune_bagging_fraction_and_bagging_freq, val_score: 1.120758:  60%|######    | 6/10 [07:51<05:03, 75.78s/it][I 2020-05-06 03:50:35,462] Finished trial#5 with value: 1.1301322055156715 with parameters: {'bagging_fraction': 0.4833483859365031, 'bagging_freq': 6}. Best is trial#2 with value: 1.1213881114040902.
tune_bagging_fraction_and_bagging_freq, val_score: 1.120758:  70%|#######   |

tune_lambda_l1_and_lambda_l2, val_score: 1.117825:  85%|########5 | 17/20 [24:03<04:14, 84.78s/it][I 2020-05-06 04:24:29,810] Finished trial#16 with value: 1.1209657750942414 with parameters: {'lambda_l1': 0.48602250865462676, 'lambda_l2': 3.288028264576887e-06}. Best is trial#13 with value: 1.1178246881442637.
tune_lambda_l1_and_lambda_l2, val_score: 1.117825:  90%|######### | 18/20 [25:33<02:52, 86.26s/it][I 2020-05-06 04:25:59,507] Finished trial#17 with value: 1.1180376893676462 with parameters: {'lambda_l1': 7.995479206473537, 'lambda_l2': 0.0003482214024248327}. Best is trial#13 with value: 1.1178246881442637.
tune_lambda_l1_and_lambda_l2, val_score: 1.117825:  95%|#########5| 19/20 [26:56<01:25, 85.35s/it][I 2020-05-06 04:27:22,755] Finished trial#18 with value: 1.1211011317652508 with parameters: {'lambda_l1': 0.0009073429945391816, 'lambda_l2': 0.0004446516501392411}. Best is trial#13 with value: 1.1178246881442637.
tune_lambda_l1_and_lambda_l2, val_score: 1.117825: 100%|#####

Best Params: {'lambda_l1': 2.972300673782286, 'lambda_l2': 0.0002499813121238942, 'num_leaves': 15, 'feature_fraction': 0.4, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 100}
CPU times: user 15h 8min 17s, sys: 2min 51s, total: 15h 11min 9s
Wall time: 1h 54min 11s


### Feature Importance

In [22]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
6,player_cnt_in_game,1386
64,changeup_pit,1406
15,strike_cnt,1406
83,height_bat,1408
14,ball_cnt,1487
84,weight_bat,1502
126,salary_dif_b-c,1528
95,salary_year_bat,1537
114,salary_dif_p-b,1548
31,point_diff,1564


In [23]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 4min 57s, sys: 1.55 s, total: 4min 59s
Wall time: 39.7 s


In [24]:
predict

array([[4.60427053e-01, 3.71875775e-02, 1.43893552e-01, ...,
        9.90332608e-06, 9.34954681e-06, 1.34422010e-01],
       [4.24132514e-01, 5.47258874e-02, 1.38351624e-01, ...,
        1.09357582e-05, 5.99049813e-06, 9.27656083e-02],
       [3.40680088e-01, 7.06498676e-02, 1.62666916e-01, ...,
        5.04745155e-05, 1.31522459e-05, 1.05539934e-01],
       ...,
       [5.14394148e-01, 5.51689896e-02, 1.14473345e-03, ...,
        3.79616001e-05, 1.60704451e-05, 3.89871448e-05],
       [5.38462899e-01, 2.35469035e-02, 6.59800177e-04, ...,
        2.70165945e-05, 1.16922525e-05, 2.39466945e-05],
       [5.10035210e-01, 2.56792875e-02, 1.16764388e-03, ...,
        2.92947376e-05, 1.14187762e-05, 2.59389373e-05]])

## 結果出力

In [25]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.460427,0.037188,0.143894,0.221903,0.002147,0.000010,0.000009,0.134422
1,1,0.424133,0.054726,0.138352,0.285739,0.004269,0.000011,0.000006,0.092766
2,2,0.340680,0.070650,0.162667,0.243491,0.076909,0.000050,0.000013,0.105540
3,3,0.419603,0.065886,0.201192,0.246025,0.012901,0.000023,0.000010,0.054360
4,4,0.276418,0.037566,0.271608,0.337248,0.012869,0.000018,0.000012,0.064262


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [26]:
submit.mean()

index    260824.500000
0             0.524188
1             0.076268
2             0.180154
3             0.053777
4             0.070871
5             0.042938
6             0.007463
7             0.044340
dtype: float64

In [27]:
#submit.to_csv('submit/tuning.csv', header=False, index=False)

### Tuning結果(10)
`
Best Params: {
    'lambda_l1': 2.972300673782286, 
    'lambda_l2': 0.0002499813121238942, 
    'num_leaves': 15, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 1.0, 
    'bagging_freq': 0, 
    'min_child_samples': 100
}
Wall time: 1h 54min 11s
`

### Tuning結果(5)
`
Best Params: {
    'lambda_l1': 6.9923570049658075, 
    'lambda_l2': 0.002378623984798833, 
    'num_leaves': 18, 
    'feature_fraction': 0.45199999999999996, 
    'bagging_fraction': 0.9799724836460725, 
    'bagging_freq': 4, 
    'min_child_samples': 20
}
`

### 評価結果
#### tuning 5 (135)
- 評価結果  : 1.3998383

#### 5 (135)
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806